# Data Collection

In [ ]:
! pip install chembl_webresource_client

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('serotonin')

In [ ]:
targets = pd.DataFrame.from_dict(target_query)
targets.pref_name.value_counts()

Serotonin 2a (5-HT2a) receptor                                       5
Serotonin 1d (5-HT1d) receptor                                       5
Serotonin 4 (5-HT4) receptor                                         5
Serotonin 1b (5-HT1b) receptor                                       4
Serotonin 3a (5-HT3a) receptor                                       4
Serotonin 7 (5-HT7) receptor                                         4
Serotonin transporter                                                4
Serotonin 2c (5-HT2c) receptor                                       3
Serotonin 2b (5-HT2b) receptor                                       3
Serotonin 2 (5-HT2) receptor                                         3
Serotonin 5a (5-HT5a) receptor                                       3
5-hydroxytryptamine receptor 1B                                      3
Serotonin 6 (5-HT6) receptor                                         3
Serotonin 3 (5-HT3) receptor                                         3
Seroto

In [ ]:
targets[targets['pref_name']=='Serotonin 2a (5-HT2a) receptor']

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
65,"[{'xref_id': 'P28223', 'xref_name': None, 'xre...",Homo sapiens,Serotonin 2a (5-HT2a) receptor,8.0,False,CHEMBL224,"[{'accession': 'P28223', 'component_descriptio...",SINGLE PROTEIN,9606
69,"[{'xref_id': 'P35363', 'xref_name': None, 'xre...",Mus musculus,Serotonin 2a (5-HT2a) receptor,8.0,False,CHEMBL5377,"[{'accession': 'P35363', 'component_descriptio...",SINGLE PROTEIN,10090
70,"[{'xref_id': 'P50129', 'xref_name': None, 'xre...",Sus scrofa,Serotonin 2a (5-HT2a) receptor,8.0,False,CHEMBL2490,"[{'accession': 'P50129', 'component_descriptio...",SINGLE PROTEIN,9823
76,"[{'xref_id': 'P14842', 'xref_name': None, 'xre...",Rattus norvegicus,Serotonin 2a (5-HT2a) receptor,8.0,False,CHEMBL322,"[{'accession': 'P14842', 'component_descriptio...",SINGLE PROTEIN,10116
78,"[{'xref_id': 'Q75Z89', 'xref_name': None, 'xre...",Bos taurus,Serotonin 2a (5-HT2a) receptor,8.0,False,CHEMBL3446,"[{'accession': 'Q75Z89', 'component_descriptio...",SINGLE PROTEIN,9913


In [ ]:
selected_target = targets.target_chembl_id[65]
selected_target

'CHEMBL224'

In [ ]:
activity = new_client.activity
result = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
compounds_dataframe = pd.DataFrame.from_dict(result)

In [ ]:
compounds_dataframe.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,80257,[],CHEMBL617392,Binding activity radioligand.,B,None,None,BAO_0000190,BAO_0000357,single protein format,CNS(=O)(=O)Cc1ccc2[nH]cc(CCN(C)C)c2c1,None,None,CHEMBL1128368,J. Med. Chem.,1995.0,"{'bei': '17.26', 'le': '0.35', 'lle': '3.78', ...",CHEMBL128,SUMATRIPTAN,CHEMBL128,5.10,False,http://www.openphacts.org/units/Nanomolar,87533,=,1,True,=,None,IC50,nM,None,7943.28,CHEMBL224,Homo sapiens,Serotonin 2a (5-HT2a) receptor,9606,None,None,pIC50,None,UO_0000065,None,5.1
1,None,89511,[],CHEMBL617393,Binding activity against 5-hydroxytryptamine 2...,B,None,None,BAO_0000190,BAO_0000019,assay format,CN(C)CCc1c[nH]c2ccc(Cn3cncn3)cc12,None,None,CHEMBL1128368,J. Med. Chem.,1995.0,"{'bei': '18.93', 'le': '0.35', 'lle': '3.19', ...",CHEMBL905,RIZATRIPTAN,CHEMBL905,5.10,False,http://www.openphacts.org/units/Nanomolar,87529,=,1,True,=,None,IC50,nM,None,7943.28,CHEMBL224,Homo sapiens,Serotonin 2a (5-HT2a) receptor,9606,None,None,pIC50,None,UO_0000065,None,5.1
2,None,284517,[],CHEMBL617355,Compound was evaluated for the affinity at 5-h...,B,None,None,BAO_0000190,BAO_0000357,single protein format,CNS(=O)(=O)Cc1ccc2[nH]cc(CCN(C)C)c2c1,None,None,CHEMBL1130196,J. Med. Chem.,1997.0,None,CHEMBL128,SUMATRIPTAN,CHEMBL128,None,False,http://www.openphacts.org/units/Nanomolar,222426,>,1,True,>,None,IC50,nM,None,10000.0,CHEMBL224,Homo sapiens,Serotonin 2a (5-HT2a) receptor,9606,None,None,IC50,nM,UO_0000065,None,10000.0
3,None,301911,[],CHEMBL617355,Compound was evaluated for the affinity at 5-h...,B,None,None,BAO_0000190,BAO_0000357,single protein format,Fc1cccc(CCN2CCN(CCCc3c[nH]c4ccc(-n5cnnc5)cc34)...,None,None,CHEMBL1130196,J. Med. Chem.,1997.0,"{'bei': '14.87', 'le': '0.27', 'lle': '2.75', ...",CHEMBL292779,None,CHEMBL292779,6.43,False,http://www.openphacts.org/units/Nanomolar,222433,=,1,True,=,None,IC50,nM,None,370.0,CHEMBL224,Homo sapiens,Serotonin 2a (5-HT2a) receptor,9606,None,None,IC50,nM,UO_0000065,None,370.0
4,None,345944,[],CHEMBL617395,Binding affinity for human cloned 5-hydroxytry...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CN(CC1CC2c3ccccc3Cc3ccccc3N2O1)C(=O)C(F)(F)F,None,None,CHEMBL1135380,Bioorg. Med. Chem. Lett.,2002.0,None,CHEMBL83220,None,CHEMBL83220,None,False,http://www.openphacts.org/units/Nanomolar,147836,<,1,True,>,None,IC50,nM,None,1000.0,CHEMBL224,Homo sapiens,Serotonin 2a (5-HT2a) receptor,9606,None,None,pIC50,None,UO_0000065,None,6.0


In [ ]:
compounds_dataframe = compounds_dataframe[compounds_dataframe.standard_value.notna()]
compounds_dataframe = compounds_dataframe[compounds_dataframe.canonical_smiles.notna()]

In [ ]:
len(compounds_dataframe)

1177

In [ ]:
len(compounds_dataframe.canonical_smiles.unique())

1102

In [ ]:
selected_columns = ['molecule_chembl_id','canonical_smiles','standard_value']
selected_data = compounds_dataframe[selected_columns]
selected_data

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL128,CNS(=O)(=O)Cc1ccc2[nH]cc(CCN(C)C)c2c1,7943.28
1,CHEMBL905,CN(C)CCc1c[nH]c2ccc(Cn3cncn3)cc12,7943.28
2,CHEMBL128,CNS(=O)(=O)Cc1ccc2[nH]cc(CCN(C)C)c2c1,10000.0
3,CHEMBL292779,Fc1cccc(CCN2CCN(CCCc3c[nH]c4ccc(-n5cnnc5)cc34)...,370.0
4,CHEMBL83220,CN(CC1CC2c3ccccc3Cc3ccccc3N2O1)C(=O)C(F)(F)F,1000.0
...,...,...,...
1971,CHEMBL4635373,COc1c(OCCOCCOCCOCCOCCOc2ccc(C3CCN(c4ccn5c(CC6C...,446.68
1972,CHEMBL4638687,COc1c(OCCOCCOCCOCCOCCOCCOc2ccc(C3CCN(c4ccn5c(C...,275.42
1973,CHEMBL4645193,COc1c(OCCOCCOCCOCCOCCOCCOCCOCCOc2ccc(C3CCN(c4c...,407.38
1974,CHEMBL4641860,COc1c(OCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOc2ccc(C3C...,302.0


In [ ]:
bioactivity_threshold = []
for i in selected_data.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
selected_data = pd.concat([selected_data, bioactivity_class], axis=1)

In [ ]:
selected_data['class'].value_counts()

active          897
intermediate    195
inactive         85
Name: class, dtype: int64

In [ ]:
selected_data

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL128,CNS(=O)(=O)Cc1ccc2[nH]cc(CCN(C)C)c2c1,7943.28,intermediate
1,CHEMBL905,CN(C)CCc1c[nH]c2ccc(Cn3cncn3)cc12,7943.28,intermediate
2,CHEMBL128,CNS(=O)(=O)Cc1ccc2[nH]cc(CCN(C)C)c2c1,10000.0,inactive
3,CHEMBL292779,Fc1cccc(CCN2CCN(CCCc3c[nH]c4ccc(-n5cnnc5)cc34)...,370.0,active
4,CHEMBL83220,CN(CC1CC2c3ccccc3Cc3ccccc3N2O1)C(=O)C(F)(F)F,1000.0,active
...,...,...,...,...
1971,CHEMBL4635373,COc1c(OCCOCCOCCOCCOCCOc2ccc(C3CCN(c4ccn5c(CC6C...,446.68,NaN
1972,CHEMBL4638687,COc1c(OCCOCCOCCOCCOCCOCCOc2ccc(C3CCN(c4ccn5c(C...,275.42,NaN
1973,CHEMBL4645193,COc1c(OCCOCCOCCOCCOCCOCCOCCOCCOc2ccc(C3CCN(c4c...,407.38,NaN
1974,CHEMBL4641860,COc1c(OCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOc2ccc(C3C...,302.0,NaN


In [ ]:
selected_data.to_csv('serotonin_5_ht_2a_receptor_bioactivity_comp.csv', index=False)

In [ ]:
! cp -r serotonin_5_ht_2a_receptor_bioactivity_comp.csv /content/drive/MyDrive/current_work/ML-Serotonin-Mol-Activity

In [ ]:
!ls -l /content/drive/MyDrive/current_work/ML-Serotonin-Mol-Activity

total 94
-rw------- 1 root root 96208 Nov 28 14:17 serotonin_5_ht_2a_receptor_bioactivity_comp.csv
